In [1]:
import pyaudio

In [3]:
a={'topic': 'aiueo'}
'topic' in a.keys()

True

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import MeCab
import random
from tqdm import tqdm

import torch
from torch import nn
import torch.optim as optim
from transformers import BertJapaneseTokenizer, BertModel, BertForMaskedLM, BertConfig
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
model_config = BertConfig.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', output_attentions=True)
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', config=model_config)

In [8]:
def bert_tokenizer(input_text):
    return tokenizer.encode(input_text, max_length=max_len, return_tensors='pt')[0]

In [12]:
class BertClassifier(nn.Module):

    def __init__(self, model):
        super(BertClassifier, self).__init__()

        # BERTモジュール
        self.bert = model  # 日本語学習済みのBERTモデル

        # headにクラス予測を追加
        # 入力はBERTの出力特徴量の次元768、出力は10クラス
        self.cls = nn.Linear(in_features=768, out_features=10)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        result = self.bert(input_ids)  # reult は、sequence_output, pooled_output

        # sequence_outputの先頭の単語ベクトルを抜き出す
        vec_0 = result[0]  # 最初の0がsequence_outputを示す
        vec_0 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語の全768要素
        vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換
        output = self.cls(vec_0)  # 全結合層

        return output, _

In [13]:
net = BertClassifier(model)

In [14]:
model_path = "/Users/jinsakuma/Downloads/model_gpu_v4.3.3.pth"
load_weights = torch.load(model_path, map_location={'cuda:0': 'cpu'})
net.load_state_dict(load_weights)

<All keys matched successfully>

In [25]:
max_len=30
device = torch.device("cpu")


# input_text="君の名はの評価は何点くらいなの"
input_text=""
token = bert_tokenizer(input_text)
token = token.unsqueeze(0)
output, attentions = net(token.to(device))
_, pred = torch.max(output, 1)

sm = torch.nn.Softmax(dim=1)
print(pred.item())

0
